In [65]:
%matplotlib qt
import mne, os, glob, pandas as pd, re

mne.set_log_level("warning")

clean = glob.glob("../data/segmented/*/clean/*-epo.fif")
dirty = [path.replace("clean", "raw") for path in clean]

zipped = list(zip(dirty, clean))

total = pd.DataFrame(columns=["dur", "sam_la", "sam_ha", "ham_la", "ham_ha", "stai_la", "stai_ha", "pre", "post", "removed"])

for pre_ar_path, post_ar_path in zipped:
    pre = mne.read_epochs(pre_ar_path)
    post = mne.read_epochs(post_ar_path)

    duration = re.search(r"/(\d+)s/", pre_ar_path).group(1)
    duration = int(duration)

    # Labeling
    meta = post.metadata
    row_with_dur = total[total["dur"] == duration]

    if len(row_with_dur) == 0:
        total.loc[len(total)] = [duration, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    total.loc[total["dur"] == duration, "sam_la"] += meta[meta["sam"].isin([0, 1])].shape[0]
    total.loc[total["dur"] == duration, "sam_ha"] += meta[meta["sam"].isin([2, 3])].shape[0]
    total.loc[total["dur"] == duration, "ham_la"] += meta[meta["ham"].isin([0, 1])].shape[0]
    total.loc[total["dur"] == duration, "ham_ha"] += meta[meta["ham"].isin([2, 3])].shape[0]

    total.loc[total["dur"] == duration, "pre"] += len(pre)
    total.loc[total["dur"] == duration, "post"] += len(post)
    total.loc[total["dur"] == duration, "removed"] += len(pre) - len(post)

df = pd.DataFrame(d)

df
total

,dur,sam_la,sam_ha,ham_la,ham_ha,stai_la,stai_ha,pre,post,removed
0,15,160,86,79,167,0,0,276,246,30
